In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/

%cd Text_ming_project/

%ls

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/Text_ming_project
apple-2023.pdf                         google-2020.pdf     microsoft-2021.pdf
en_core_web_sm-3.8.0-py3-none-any.whl  google-2021.pdf     microsoft-2022.pdf
goog-10-k-2023.pdf                     google-2022.pdf     microsoft-2023.pdf
google-2017.pdf                        microsoft-2018.pdf  requirements.txt
google-2018.pdf                        microsoft-2019.pdf  Text_mining_project.ipynb
google-2019.pdf                        microsoft-2020.pdf


In [ ]:
%pip install pdfplumber PyPDF2
%pip install spacy
%pip wheel https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl
%pip install bertopic
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.5 MB/s eta 0:00:00
  File was already downloaded /content/drive/MyDrive/Text_ming_project/en_core_web_sm-3.8.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.2 MB/s eta 0:00:00
✔ Download and installat

In [ ]:
import pdfplumber
import re
from PyPDF2 import PdfReader, PdfWriter
import matplotlib.pyplot as plt
import spacy

In [ ]:
def page_extract(pdf_path, chunk_size):
  with pdfplumber.open(pdf_path) as pdf:

    toc_page = None #finding the table of content page

    for _, page in enumerate(pdf.pages):
        text = page.extract_text()
        if re.search(r'\btable of contents|index\b', text.lower(), re.IGNORECASE):
            toc_page = text
            break

  section = "Mine Safety Disclosures" #the last subheading that we want to get all the information from the 10Q form

  #print(toc_page)
  # To capture the page number right after the header
  part_i_page = re.search(rf"{section}.*?(\d+)", toc_page)

  count = int(part_i_page.group(1))

  reader = PdfReader(pdf_path)
  pdf_writer = PdfWriter()

  content = []

  for i in range(count + 2):
      a = pdf_writer.add_page(reader.pages[i]).extract_text()
      content.append(a)


  full_text = " ".join(content)

  words = full_text.split()  # Split the text into words
  chunks = []
  chunk = []

  current_size = 0

  for word in words:
        word_length = len(word) + 1
        if current_size + word_length > chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
            current_size = 0
        chunk.append(word)
        current_size += word_length


  if chunk:
        chunks.append(" ".join(chunk))

  return chunks


nlp = spacy.load('en_core_web_sm')

def pre_process(content):
    preprocess = []
    for i in range(len(content)):
      text = re.sub(r'\d+', '', content[i])
      text = re.sub(r'-', ' ', text)
      text = re.sub(r'[^\w\s]', '', text)
      text = re.sub(r'\s+', ' ', text).strip()
      text = re.sub(r'_+', '', text)
      text = text.lower()

      doc = nlp(text)

      #lemmatized_text = [token.lemma_ for token in doc]

      final = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

      preprocess.append(" ".join(final))

    return preprocess


from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

def train_lda_model(documents, num_topics, passes=1):
    dictionary = Dictionary(documents)

    corpus = [dictionary.doc2bow(doc) for doc in documents]
    dictionary[0]
    model = LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=num_topics, passes=passes)
    return model

2017

In [ ]:
microsoft_2017 = page_extract("microsoft-2017.pdf",chunk_size=500)
microsoft_2017_process = pre_process(microsoft_2017)

lda_content_2017 = [content.split() for content in microsoft_2017_process]
model_2017 = train_lda_model(lda_content_2017,num_topics=10,passes=10)
model_2017.print_topics()

[(0,
  '0.013*"license" + 0.012*"business" + 0.009*"share" + 0.009*"purchase" + 0.009*"organization" + 0.009*"item" + 0.008*"year" + 0.007*"operation" + 0.007*"software" + 0.007*"service"'),
 (1,
  '0.025*"device" + 0.020*"product" + 0.020*"window" + 0.013*"oem" + 0.012*"change" + 0.011*"service" + 0.010*"business" + 0.009*"new" + 0.009*"include" + 0.008*"revenue"'),
 (2,
  '0.014*"company" + 0.013*"product" + 0.012*"provide" + 0.011*"compete" + 0.011*"competition" + 0.010*"dynamic" + 0.010*"solution" + 0.010*"include" + 0.009*"service" + 0.009*"offering"'),
 (3,
  '0.013*"organization" + 0.013*"sale" + 0.012*"significant" + 0.011*"risk" + 0.008*"market" + 0.008*"revenue" + 0.007*"global" + 0.007*"research" + 0.007*"affect" + 0.007*"adversely"'),
 (4,
  '0.016*"product" + 0.013*"service" + 0.010*"law" + 0.010*"datum" + 0.009*"tax" + 0.009*"software" + 0.008*"sale" + 0.008*"customer" + 0.007*"organization" + 0.006*"jurisdiction"'),
 (5,
  '0.030*"item" + 0.014*"service" + 0.009*"cloud" 

In [ ]:
from bertopic import BERTopic

topic_model_2017 = BERTopic()
topics, probs = topic_model_2017.fit_transform(microsoft_2017_process)

topic_model_2017.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,101,"[product, business, service, software, revenue..."
1,0,38,"[item, financial, statement, income, report, r..."
2,1,38,"[cloud, service, business, server, application..."
3,2,20,"[datum, change, law, regulation, result, pract..."
4,3,16,"[device, window, platform, oem, application, h..."
5,4,15,"[license, organization, agreement, software, p..."
6,5,12,"[security, customer, service, threat, product,..."


2018

In [ ]:
microsoft_2018 = page_extract("microsoft-2018.pdf",chunk_size=500)
microsoft_2018_process = pre_process(microsoft_2018)

lda_content_2018 = [content.split() for content in microsoft_2018_process]
model_2018 = train_lda_model(lda_content_2018,num_topics=10,passes=10)
model_2018.print_topics()

[(0,
  '0.021*"device" + 0.012*"window" + 0.010*"revenue" + 0.010*"ai" + 0.009*"employee" + 0.009*"brand" + 0.009*"experience" + 0.008*"new" + 0.008*"business" + 0.007*"ability"'),
 (1,
  '0.017*"business" + 0.011*"product" + 0.010*"operating" + 0.009*"sale" + 0.008*"system" + 0.007*"research" + 0.007*"development" + 0.007*"support" + 0.006*"new" + 0.006*"segment"'),
 (2,
  '0.023*"product" + 0.012*"service" + 0.012*"customer" + 0.010*"business" + 0.010*"change" + 0.009*"technology" + 0.008*"new" + 0.008*"security" + 0.008*"result" + 0.007*"include"'),
 (3,
  '0.011*"executive" + 0.010*"application" + 0.009*"cloud" + 0.009*"officer" + 0.008*"product" + 0.008*"service" + 0.008*"consumer" + 0.008*"opportunity" + 0.007*"customer" + 0.007*"developer"'),
 (4,
  '0.024*"service" + 0.022*"product" + 0.016*"business" + 0.014*"office" + 0.014*"server" + 0.013*"microsoft" + 0.013*"president" + 0.013*"include" + 0.011*"software" + 0.010*"license"'),
 (5,
  '0.028*"service" + 0.016*"product" + 0.0

In [ ]:
topic_model_2018 = BERTopic()
topics, probs = topic_model_2018.fit_transform(microsoft_2018_process)

topic_model_2018.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,55,"[president, vice, business, service, product, ..."
1,0,68,"[service, device, product, license, window, so..."
2,1,58,"[datum, product, customer, service, security, ..."
3,2,41,"[item, financial, statement, tax, form, act, r..."
4,3,33,"[cloud, ai, service, business, azure, base, de..."


2019

In [ ]:
microsoft_2019 = page_extract("microsoft-2019.pdf",chunk_size=500)
microsoft_2019_process = pre_process(microsoft_2019)

lda_content_2019 = [content.split() for content in microsoft_2019_process]
model_2019 = train_lda_model(lda_content_2019,num_topics=10,passes=10)
model_2019.print_topics()

[(0,
  '0.013*"license" + 0.013*"security" + 0.012*"agreement" + 0.010*"registrant" + 0.009*"organization" + 0.009*"purchase" + 0.008*"open" + 0.008*"exchange" + 0.008*"business" + 0.008*"indicate"'),
 (1,
  '0.018*"service" + 0.014*"item" + 0.012*"include" + 0.012*"business" + 0.010*"base" + 0.009*"customer" + 0.008*"financial" + 0.007*"cloud" + 0.007*"provide" + 0.007*"billion"'),
 (2,
  '0.021*"product" + 0.013*"service" + 0.010*"security" + 0.009*"software" + 0.009*"system" + 0.008*"compete" + 0.008*"customer" + 0.007*"market" + 0.007*"device" + 0.006*"business"'),
 (3,
  '0.018*"service" + 0.017*"product" + 0.014*"business" + 0.010*"datum" + 0.008*"company" + 0.008*"cloud" + 0.008*"item" + 0.008*"financial" + 0.007*"new" + 0.007*"customer"'),
 (4,
  '0.031*"service" + 0.017*"microsoft" + 0.016*"window" + 0.016*"license" + 0.016*"product" + 0.016*"software" + 0.014*"organization" + 0.013*"customer" + 0.012*"cloud" + 0.011*"include"'),
 (5,
  '0.012*"include" + 0.010*"office" + 0.01

In [ ]:
topic_model_2019 = BERTopic()
topics, probs = topic_model_2019.fit_transform(microsoft_2019_process)

topic_model_2019.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,54,"[president, vice, product, business, executive..."
1,0,55,"[service, cloud, new, office, business, produc..."
2,1,47,"[item, financial, statement, result, share, ex..."
3,2,32,"[security, customer, user, product, service, d..."
4,3,23,"[device, platform, application, window, compet..."
5,4,20,"[law, datum, regulation, compliance, eu, oblig..."
6,5,15,"[intellectual, property, patent, right, licens..."
7,6,14,"[license, organization, agreement, service, so..."


2020

In [ ]:
microsoft_2020 = page_extract("microsoft-2020.pdf",chunk_size=500)
microsoft_2020_process = pre_process(microsoft_2020)

lda_content_2020 = [content.split() for content in microsoft_2020_process]
model_2020 = train_lda_model(lda_content_2020,num_topics=10,passes=10)
model_2020.print_topics()

[(0,
  '0.015*"employee" + 0.015*"president" + 0.013*"customer" + 0.013*"marketing" + 0.011*"microsoft" + 0.010*"executive" + 0.010*"vice" + 0.009*"officer" + 0.008*"company" + 0.008*"office"'),
 (1,
  '0.013*"business" + 0.011*"market" + 0.008*"share" + 0.007*"company" + 0.007*"customer" + 0.007*"employee" + 0.006*"global" + 0.006*"growth" + 0.006*"claim" + 0.006*"cloud"'),
 (2,
  '0.024*"service" + 0.018*"device" + 0.016*"business" + 0.014*"product" + 0.013*"office" + 0.012*"include" + 0.011*"customer" + 0.010*"cloud" + 0.010*"application" + 0.009*"user"'),
 (3,
  '0.015*"item" + 0.014*"report" + 0.011*"financial" + 0.010*"president" + 0.008*"executive" + 0.008*"vice" + 0.008*"result" + 0.008*"officer" + 0.008*"file" + 0.008*"risk"'),
 (4,
  '0.027*"product" + 0.020*"license" + 0.018*"agreement" + 0.015*"service" + 0.014*"organization" + 0.010*"technology" + 0.010*"revenue" + 0.009*"software" + 0.009*"intellectual" + 0.009*"property"'),
 (5,
  '0.013*"content" + 0.013*"item" + 0.013*

In [ ]:
topic_model_2020 = BERTopic()
topics, probs = topic_model_2020.fit_transform(microsoft_2020_process)

topic_model_2020.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,75,"[product, service, device, software, developme..."
1,0,134,"[product, result, business, service, customer,..."
2,1,19,"[cloud, azure, service, edge, intelligent, ai,..."
3,2,17,"[license, organization, agreement, service, mi..."
4,3,16,"[game, gaming, device, xbox, content, console,..."
5,4,15,"[solution, talent, help, linkedin, business, t..."


2021

In [ ]:
microsoft_2021 = page_extract("microsoft-2021.pdf",chunk_size=500)
microsoft_2021_process = pre_process(microsoft_2021)

lda_content_2021 = [content.split() for content in microsoft_2021_process]
model_2021 = train_lda_model(lda_content_2021,num_topics=10,passes=10)
model_2021.print_topics()

[(0,
  '0.014*"product" + 0.014*"service" + 0.011*"business" + 0.011*"include" + 0.010*"development" + 0.010*"device" + 0.009*"content" + 0.008*"system" + 0.008*"software" + 0.008*"platform"'),
 (1,
  '0.020*"service" + 0.020*"microsoft" + 0.017*"organization" + 0.014*"license" + 0.013*"agreement" + 0.011*"president" + 0.009*"vice" + 0.009*"customer" + 0.009*"serve" + 0.008*"mr"'),
 (2,
  '0.014*"employee" + 0.011*"service" + 0.009*"user" + 0.008*"product" + 0.008*"help" + 0.007*"information" + 0.007*"time" + 0.007*"cloud" + 0.007*"community" + 0.007*"security"'),
 (3,
  '0.029*"service" + 0.021*"product" + 0.013*"revenue" + 0.012*"device" + 0.011*"office" + 0.010*"software" + 0.010*"microsoft" + 0.010*"new" + 0.010*"oem" + 0.010*"application"'),
 (4,
  '0.018*"service" + 0.016*"product" + 0.010*"gaming" + 0.010*"executive" + 0.010*"include" + 0.009*"president" + 0.009*"business" + 0.008*"vice" + 0.008*"game" + 0.008*"platform"'),
 (5,
  '0.015*"solution" + 0.014*"company" + 0.012*"cus

In [ ]:
topic_model_2021 = BERTopic()
topics, probs = topic_model_2021.fit_transform(microsoft_2021_process)

topic_model_2021.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,95,"[product, business, include, service, presiden..."
1,0,41,"[employee, learn, work, microsoft, solution, t..."
2,1,36,"[cloud, ai, service, azure, business, device, ..."
3,2,30,"[item, financial, statement, report, registran..."
4,3,30,"[product, application, device, platform, softw..."
5,4,30,"[law, compliance, regulation, datum, action, e..."
6,5,28,"[security, customer, service, product, datum, ..."
7,6,13,"[license, organization, agreement, service, so..."
8,7,12,"[game, gaming, xbox, content, console, cloud, ..."


Even though "Covid" is not mentioned via Bertopic, it was foufn it topic-8 via LDA for 2021 as shown above ^

2022

In [ ]:
microsoft_2022 = page_extract("microsoft-2022.pdf",chunk_size=500)
microsoft_2022_process = pre_process(microsoft_2022)

lda_content_2022 = [content.split() for content in microsoft_2022_process]
model_2022 = train_lda_model(lda_content_2022,num_topics=10,passes=10)
model_2022.print_topics()

[(0,
  '0.010*"customer" + 0.009*"business" + 0.008*"product" + 0.007*"system" + 0.006*"application" + 0.006*"datacenter" + 0.006*"learn" + 0.006*"support" + 0.006*"security" + 0.006*"include"'),
 (1,
  '0.019*"business" + 0.011*"item" + 0.008*"include" + 0.008*"property" + 0.008*"employee" + 0.007*"information" + 0.007*"customer" + 0.007*"report" + 0.006*"increase" + 0.006*"brand"'),
 (2,
  '0.013*"include" + 0.011*"product" + 0.011*"window" + 0.011*"device" + 0.011*"service" + 0.010*"revenue" + 0.008*"system" + 0.008*"increase" + 0.008*"impact" + 0.008*"cost"'),
 (3,
  '0.015*"employee" + 0.011*"support" + 0.010*"item" + 0.009*"service" + 0.008*"development" + 0.008*"microsoft" + 0.007*"gaming" + 0.006*"research" + 0.005*"azure" + 0.005*"survey"'),
 (4,
  '0.019*"service" + 0.017*"customer" + 0.012*"license" + 0.012*"microsoft" + 0.012*"agreement" + 0.012*"product" + 0.011*"organization" + 0.009*"software" + 0.009*"technology" + 0.007*"item"'),
 (5,
  '0.027*"service" + 0.025*"produc

In [ ]:
topic_model_2022 = BERTopic()
topics, probs = topic_model_2022.fit_transform(microsoft_2022_process)

topic_model_2022.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,62,"[president, service, microsoft, vice, business..."
1,0,94,"[item, result, financial, market, law, risk, p..."
2,1,45,"[device, product, window, game, oem, content, ..."
3,2,36,"[employee, learn, support, work, manager, cult..."
4,3,35,"[cloud, service, azure, ai, business, applicat..."
5,4,33,"[security, customer, service, product, attack,..."
6,5,16,"[license, agreement, organization, solution, s..."


2023

In [ ]:
microsoft_2023 = page_extract("microsoft-2023.pdf",chunk_size=800)
microsoft_2023_process = pre_process(microsoft_2023)

lda_content_2023 = [content.split() for content in microsoft_2023_process]
model_2023 = train_lda_model(lda_content_2023,num_topics=10,passes=10)
model_2023.print_topics()

[(0,
  '0.022*"ai" + 0.017*"microsoft" + 0.015*"new" + 0.011*"business" + 0.011*"product" + 0.010*"work" + 0.009*"create" + 0.008*"help" + 0.008*"value" + 0.008*"technology"'),
 (1,
  '0.014*"business" + 0.010*"result" + 0.010*"risk" + 0.009*"change" + 0.009*"country" + 0.009*"segment" + 0.008*"financial" + 0.007*"supply" + 0.007*"regulatory" + 0.007*"include"'),
 (2,
  '0.013*"microsoft" + 0.012*"serve" + 0.011*"president" + 0.010*"company" + 0.010*"vice" + 0.009*"ms" + 0.009*"officer" + 0.008*"law" + 0.007*"business" + 0.007*"employee"'),
 (3,
  '0.021*"service" + 0.020*"product" + 0.016*"software" + 0.013*"device" + 0.013*"business" + 0.013*"system" + 0.012*"include" + 0.012*"security" + 0.009*"operating" + 0.009*"customer"'),
 (4,
  '0.019*"agreement" + 0.017*"license" + 0.015*"organization" + 0.011*"customer" + 0.011*"service" + 0.009*"microsoft" + 0.008*"include" + 0.008*"partner" + 0.007*"purchase" + 0.007*"percent"'),
 (5,
  '0.012*"microsoft" + 0.012*"president" + 0.011*"servi

In [ ]:
topic_model_2023 = BERTopic()
topics, probs = topic_model_2023.fit_transform(microsoft_2023_process)

topic_model_2023.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,33,"[president, vice, business, service, executive..."
1,0,96,"[service, product, cloud, microsoft, device, p..."
2,1,28,"[law, datum, result, product, eu, regulation, ..."
3,2,21,"[item, report, financial, statement, act, sect..."
4,3,15,"[security, attack, customer, service, product,..."


trend across years

In [ ]:
import pandas as pd

data = {2017: microsoft_2017_process,2018:microsoft_2018_process, 2019: microsoft_2019_process, 2020: microsoft_2020_process, 2021: microsoft_2021_process, 2022: microsoft_2022_process, 2023: microsoft_2023_process}

def create_yearly_documents(data):
    documents = []

    for year, doc_list in data.items():
        for doc in doc_list:
            documents.append((doc, year))

    return documents

documents = create_yearly_documents(data)

# Convert the list of tuples into a DataFrame
df = pd.DataFrame(documents, columns=['text', 'year'])

print(df)

                                                   text  year
0     united states securities exchange commission w...  2017
1     wwwmicrosoftcominvestor security register purs...  2017
2     indicate check mark registrant fi lead report ...  2017
3     interactive data file require submit post purs...  2017
4     information statement incor porate reference i...  2017
...                                                 ...   ...
1855  staff security exchange commission issue day p...  2023
1856  largest own international property include spa...  2023
1857  internationally june square foot million locat...  2023
1858  repurchase fourth quarter fiscal year period t...  2023
1859  award board director declare follow dividend f...  2023

[1860 rows x 2 columns]


In [ ]:
topic_model = BERTopic()

text = df['text'].tolist()

topics, probs = topic_model.fit_transform(text)

timestamps = df["year"].to_list()

topics_over_time = topic_model.topics_over_time(text, timestamps)

# Visualize topics over time
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topics_of_interest = [1,2,4,6,32,34,39,48]

topic_model.visualize_topics_over_time(topics_over_time, topics = topics_of_interest)